In [52]:
import pandas as pd
import numpy as np
import math
import datetime

table = pd.read_csv("../Dati/orari/route.csv", delimiter=";")
time_table = pd.read_csv("../Dati/orari/route_time.csv", delimiter=";")
dft = pd.read_csv("../Dati/distanze/times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")
fur2 = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";")

def get_row(df, string):
    return pd.Series(df.index).str.contains(string, case=False)

def get_column(df, string):
    return pd.Series(df.columns).str.contains(string, case=False)

def difference(day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    return ((time2-time).seconds)/60

#TOFIX
def difference2(day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    if(time > datetime.datetime.strptime('12:00', '%H:%M')):
        time = time + datetime.datetime.strptime('01:00', '%H:%M')
        print(time)
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    return ((time2-time).seconds)/60

In [46]:

#start = dft[get_row(dft, 'Cremona, sesto 39').values]
#day='lunedi'

#indici

mean = []
#Lista dei paesi non raggiungibili
impossibile = []
for day in ['lunedi', 'venerdi']:
    row = dft[get_row(dft, table[day][0]).values]
    i1 = 0
    biases = []
    sumb = 0
    for i in range(1,len(time_table[day].dropna())):
        paese=table[day][i]
        #print(paese)
        #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
        value=row.loc[:,get_column(dft, paese).values]
        row = dft[get_row(dft, table[day][i]).values]

        diff=difference(day, i1, i)
        print(f"diff: {diff} , value: {value.values[0][0]}")
        b = diff-value.values[0][0]
        biases.append(b)
        sumb+=diff-value.values[0][0]
        i1 = i
    mean.append(sumb/len(biases))
print(mean)
#15 minuti per carico e scarico (e caffé)

diff: 10.0 , value: 5.58
diff: 20.0 , value: 13.05
diff: 15.0 , value: 16.97
diff: 15.0 , value: 4.88
diff: 20.0 , value: 17.72
diff: 15.0 , value: 12.62
diff: 10.0 , value: 5.97
diff: 25.0 , value: 21.95
diff: 80.0 , value: 11.3
diff: 40.0 , value: 10.33
diff: 30.0 , value: 23.88
diff: 25.0 , value: 6.67
diff: 25.0 , value: 16.32
diff: 35.0 , value: 11.8
diff: 20.0 , value: 10.82
diff: 15.0 , value: 9.8
diff: 20.0 , value: 9.57
diff: 20.0 , value: 15.33
diff: 15.0 , value: 12.77
diff: 70.0 , value: 13.92
diff: 30.0 , value: 10.22
diff: 25.0 , value: 14.98
diff: 15.0 , value: 13.78
diff: 15.0 , value: 10.6
[12.52, 13.31]


In [45]:
for day in ['lunedi', 'venerdi']:
    schedule = fur1[day]
    row = dft[get_row(dft, schedule[0]).values]
    i1 = 0
    diff_greedy = []
    diff_curr = []
    for j in range(1, len(schedule.dropna())):
        paese = schedule[j]
        value = row.loc[:,get_column(dft, paese).values].values[0][0] + 13
        diff_greedy.append(value)
        diff=difference(day, i1, j)
        diff_curr.append(diff)
        i1 = j
    print(day)
    for j in range(len(schedule.dropna())-1):
        print(f" diff greedy: {diff_greedy[j]} , diff current: {diff_curr[j]}")

lunedi
 diff greedy: 18.58 , diff current: 10.0
 diff greedy: 26.55 , diff current: 20.0
 diff greedy: 33.730000000000004 , diff current: 15.0
 diff greedy: 37.67 , diff current: 15.0
 diff greedy: 51.2 , diff current: 20.0
 diff greedy: 45.88 , diff current: 15.0
 diff greedy: 46.52 , diff current: 10.0
 diff greedy: 32.370000000000005 , diff current: 25.0
 diff greedy: 29.23 , diff current: 80.0
 diff greedy: 28.27 , diff current: 40.0
 diff greedy: 43.05 , diff current: 30.0
 diff greedy: 46.57 , diff current: 25.0
venerdi
 diff greedy: 24.8 , diff current: 35.0
 diff greedy: 29.5 , diff current: 20.0
 diff greedy: 28.07 , diff current: 15.0
 diff greedy: 28.22 , diff current: 20.0
 diff greedy: 22.18 , diff current: 20.0
 diff greedy: 24.35 , diff current: 15.0
 diff greedy: 24.15 , diff current: 70.0
 diff greedy: 28.33 , diff current: 30.0
 diff greedy: 33.769999999999996 , diff current: 25.0
 diff greedy: 27.630000000000003 , diff current: 15.0


In [53]:
difference2('lunedi', 8, 9)

TypeError: unsupported operand type(s) for +: 'datetime.datetime' and 'datetime.datetime'